# NiN

- 全连接的问题
- -卷积层需要较少的参数$c_i \times c_o \times k^2$
- 但卷积层后的第一个全连接层的参数
  - LeNet   16x5x5x120=48k
  - AlexNet 256x5x5x4096=26M
  - VGG  512x7x7x4096=102M

## NiN块
- 一个卷积层后跟两个全连接层
  - 步幅1，无填充，输出形状跟卷积层输出一样
  - 起到全连接层的作用

## NiN架构
- 无全连接层
- 交替使用NiN块和步幅为2的最大池化层
  - 逐步减少高宽和增大通道数
- 最后使用全局平均池化层得到输出
  - 其输入通道数是类别数

## 